Simple GUI Test
===
Use ChatGPT to build a basic working version

# Basic GUI

In [1]:
import tkinter as tk
from tkinter import ttk

class PCBuilderApp(tk.Tk):
    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        self.title("PC Builder")
        self.geometry("400x400")

        # Sample data
        self.gpus = {
            "Low End GPU": ["Low End CPU 1", "Low End CPU 2"],
            "Mid Range GPU": ["Mid Range CPU 1", "Mid Range CPU 2"],
            "High End GPU": ["High End CPU 1", "High End CPU 2"]
        }

        self.cpus = {
            "Low End CPU 1": ["Low End Motherboard 1", "Low End Motherboard 2"],
            "Low End CPU 2": ["Low End Motherboard 3", "Low End Motherboard 4"],
            "Mid Range CPU 1": ["Mid Range Motherboard 1", "Mid Range Motherboard 2"],
            "Mid Range CPU 2": ["Mid Range Motherboard 3", "Mid Range Motherboard 4"],
            "High End CPU 1": ["High End Motherboard 1", "High End Motherboard 2"],
            "High End CPU 2": ["High End Motherboard 3", "High End Motherboard 4"]
        }

        # GPU Label and Dropdown
        gpu_label = ttk.Label(self, text="Select GPU:")
        gpu_label.pack(pady=5)
        self.gpu_var = tk.StringVar(self)
        self.gpu_var.set(list(self.gpus.keys())[0])  # set the default option
        self.gpu_menu = ttk.Combobox(self, textvariable=self.gpu_var, values=list(self.gpus.keys()))
        self.gpu_menu.pack(pady=5)
        self.gpu_menu.bind("<<ComboboxSelected>>", self.update_cpus)

        # CPU Label and Dropdown
        cpu_label = ttk.Label(self, text="Select CPU:")
        cpu_label.pack(pady=5)
        self.cpu_var = tk.StringVar(self)
        self.cpu_var.set(self.gpus[self.gpu_var.get()][0])  # set the default option
        self.cpu_menu = ttk.Combobox(self, textvariable=self.cpu_var, values=self.gpus[self.gpu_var.get()])
        self.cpu_menu.pack(pady=5)
        self.cpu_menu.bind("<<ComboboxSelected>>", self.update_motherboards)

        # Motherboard Label and Dropdown
        motherboard_label = ttk.Label(self, text="Select Motherboard:")
        motherboard_label.pack(pady=5)
        self.motherboard_var = tk.StringVar(self)
        self.motherboard_var.set(self.cpus[self.cpu_var.get()][0])  # set the default option
        self.motherboard_menu = ttk.Combobox(self, textvariable=self.motherboard_var, values=self.cpus[self.cpu_var.get()])
        self.motherboard_menu.pack(pady=5)

        # Button to display selected components
        build_button = ttk.Button(self, text="Build PC", command=self.display_selected_components)
        build_button.pack(pady=10)

        # Text box to display selected components
        self.result_text = tk.Text(self, height=5, width=40)
        self.result_text.pack(pady=10)

    def update_cpus(self, event):
        gpu = self.gpu_var.get()
        cpus = self.gpus[gpu]
        self.cpu_var.set(cpus[0])  # set the default option
        self.cpu_menu['values'] = cpus
        self.update_motherboards(None)  # update motherboards for the default CPU

    def update_motherboards(self, event):
        cpu = self.cpu_var.get()
        motherboards = self.cpus[cpu]
        self.motherboard_var.set(motherboards[0])  # set the default option
        self.motherboard_menu['values'] = motherboards

    def display_selected_components(self):
        gpu = self.gpu_var.get()
        cpu = self.cpu_var.get()
        motherboard = self.motherboard_var.get()
        result = f"GPU: {gpu}\nCPU: {cpu}\nMotherboard: {motherboard}"
        self.result_text.delete(1.0, tk.END)
        self.result_text.insert(tk.END, result)

def main():
    app = PCBuilderApp()
    app.mainloop()

if __name__ == "__main__":
    main()


# Modularized GUI

In [1]:
# main.py
import tkinter as tk
from tkinter import ttk
import json

class Dropdown:
    def __init__(self, master, label_text, options, update_func):
        self.label = ttk.Label(master, text=label_text)
        self.label.pack(pady=5)
        self.var = tk.StringVar(master)
        self.var.set(options[0])  # set the default option
        self.menu = ttk.Combobox(master, textvariable=self.var, values=options)
        self.menu.pack(pady=5)
        self.menu.bind("<<ComboboxSelected>>", update_func)

class Updater:
    def __init__(self, data, from_dropdown, to_dropdown, next_updater=None):
        self.data = data
        self.from_dropdown = from_dropdown
        self.to_dropdown = to_dropdown
        self.next_updater = next_updater

    def update(self, event):
        key = self.from_dropdown.var.get()
        values = self.data[key]
        self.to_dropdown.var.set(values[0])  # set the default option
        self.to_dropdown.menu['values'] = values
        if self.next_updater is not None:
            self.next_updater.update(None)

class PCBuilderApp(tk.Tk):
    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        self.title("PC DIY Helper")
        self.geometry("400x400")

        # Load configuration
        with open('config.json') as f:
            config = json.load(f)
        self.cpus = config['CPUs']
        self.gpus = config['GPUs']

        # GPU Dropdown
        self.gpu_dropdown = Dropdown(self, "Select GPU:", list(self.gpus.keys()), None)

        # CPU Dropdown
        self.cpu_dropdown = Dropdown(self, "Select CPU:", self.gpus[self.gpu_dropdown.var.get()], None)

        # Motherboard Dropdown
        self.motherboard_dropdown = Dropdown(self, "Select Motherboard:", self.cpus[self.cpu_dropdown.var.get()], self.do_nothing)

        # Updaters
        self.motherboard_updater = Updater(self.cpus, self.cpu_dropdown, self.motherboard_dropdown)
        self.cpu_updater = Updater(self.gpus, self.gpu_dropdown, self.cpu_dropdown, self.motherboard_updater)

        # Set update functions for dropdowns
        self.gpu_dropdown.menu.bind("<<ComboboxSelected>>", self.cpu_updater.update)
        self.cpu_dropdown.menu.bind("<<ComboboxSelected>>", self.motherboard_updater.update)

        # Button to display selected components
        build_button = ttk.Button(self, text="Build PC", command=self.display_selected_components)
        build_button.pack(pady=10)

        # Text box to display selected components
        self.result_text = tk.Text(self, height=5, width=40)
        self.result_text.pack(pady=10)

    def display_selected_components(self):
        gpu = self.gpu_dropdown.var.get()
        cpu = self.cpu_dropdown.var.get()
        motherboard = self.motherboard_dropdown.var.get()
        result = f"GPU: {gpu}\nCPU: {cpu}\nMotherboard: {motherboard}"
        self.result_text.delete(1.0, tk.END)
        self.result_text.insert(tk.END, result)

    def do_nothing(self, event):
        pass

def main():
    app = PCBuilderApp()
    app.mainloop()

if __name__ == "__main__":
    main()


## Add more components

In [ ]:
import tkinter as tk
from tkinter import ttk
import json

class Dropdown:
    def __init__(self, master, label_text, options, update_func, row, col):
        self.label = ttk.Label(master, text=label_text)
        self.label.grid(row=row, column=col, padx=5, pady=5)
        self.var = tk.StringVar(master)
        self.var.set(options[0])  # set the default option
        self.menu = ttk.Combobox(master, textvariable=self.var, values=options)
        self.menu.grid(row=row + 1, column=col, padx=5, pady=5)
        self.menu.bind("<<ComboboxSelected>>", update_func)

class Updater:
    def __init__(self, data, from_dropdown, to_dropdowns, next_updaters=None):
        self.data = data
        self.from_dropdown = from_dropdown
        self.to_dropdowns = to_dropdowns
        self.next_updaters = next_updaters

    def update(self, event):
        key = self.from_dropdown.var.get()
        values = self.data[key]
        for to_dropdown in self.to_dropdowns:
            to_dropdown.var.set(values[0])  # set the default option
            to_dropdown.menu['values'] = values
        if self.next_updaters is not None:
            for next_updater in self.next_updaters:
                next_updater.update(None)

class PCBuilderApp(tk.Tk):
    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        self.title("PC DIY Helper")
        self.geometry("800x600")

        # Load configuration
        with open('config.json') as f:
            config = json.load(f)
        self.config = config

        # GPU Dropdown
        self.gpu_dropdown = Dropdown(self, "Select GPU:", list(config['GPUs'].keys()), None, 0, 0)

        # CPU Dropdown
        self.cpu_dropdown = Dropdown(self, "Select CPU:", config['GPUs'][self.gpu_dropdown.var.get()], None, 1, 0)

        # Motherboard Dropdown
        self.motherboard_dropdown = Dropdown(self, "Select Motherboard:", config['CPUs'][self.cpu_dropdown.var.get()], self.do_nothing, 2, 0)

        # RAM Dropdown
        self.ram_dropdown = Dropdown(self, "Select RAM:", config['Motherboards'][self.motherboard_dropdown.var.get()], self.do_nothing, 3, 0)

        # SSD Dropdown
        self.ssd_dropdown = Dropdown(self, "Select SSD:", config['Motherboards'][self.motherboard_dropdown.var.get()], self.do_nothing, 4, 0)

        # Cooling System Dropdown
        self.cooling_dropdown = Dropdown(self, "Select Cooling System:", config['Motherboards'][self.motherboard_dropdown.var.get()], self.do_nothing, 5, 0)

        # Fans Dropdown
        self.fans_dropdown = Dropdown(self, "Select Fans:", config['Motherboards'][self.motherboard_dropdown.var.get()], self.do_nothing, 6, 0)

        # PSU Dropdown
        self.psu_dropdown = Dropdown(self, "Select PSU:", config['Motherboards'][self.motherboard_dropdown.var.get()], self.do_nothing, 7, 0)

        # Case Dropdown
        self.case_dropdown = Dropdown(self, "Select Case:", config['Motherboards'][self.motherboard_dropdown.var.get()], self.do_nothing, 8, 0)

        # Updaters
        self.case_updater = Updater(config['Cases'], self.psu_dropdown, [self.case_dropdown], None)
        self.psu_updater = Updater(config['PSUs'], self.fans_dropdown, [self.psu_dropdown], [self.case_updater])
        self.fans_updater = Updater(config['Fans'], self.cooling_dropdown, [self.fans_dropdown], [self.psu_updater])
        self.cooling_updater = Updater(config['Coolings'], self.ssd_dropdown, [self.cooling_dropdown], [self.fans_updater])
        self.ssd_updater = Updater(config['SSDs'], self.ram_dropdown, [self.ssd_dropdown], [self.cooling_updater])
        self.ram_updater = Updater(config['RAMs'], self.motherboard_dropdown, [self.ram_dropdown], [self.ssd_updater])
        self.motherboard_updater = Updater(config['Motherboards'], self.cpu_dropdown, [self.motherboard_dropdown, self.ram_dropdown, self.ssd_dropdown, self.cooling_dropdown, self.fans_dropdown, self.psu_dropdown, self.case_dropdown], [self.ram_updater, self.ssd_updater, self.cooling_updater, self.fans_updater, self.psu_updater, self.case_updater])
        self.cpu_updater = Updater(config['CPUs'], self.gpu_dropdown, [self.cpu_dropdown, self.motherboard_dropdown, self.ram_dropdown, self.ssd_dropdown, self.cooling_dropdown, self.fans_dropdown, self.psu_dropdown, self.case_dropdown], [self.motherboard_updater, self.ram_updater, self.ssd_updater, self.cooling_updater, self.fans_updater, self.psu_updater, self.case_updater])

        # Set update functions for dropdowns
        self.gpu_dropdown.menu.bind("<<ComboboxSelected>>", self.cpu_updater.update)
        self.cpu_dropdown.menu.bind("<<ComboboxSelected>>", self.motherboard_updater.update)
        self.motherboard_dropdown.menu.bind("<<ComboboxSelected>>", self.ram_updater.update)
        self.ram_dropdown.menu.bind("<<ComboboxSelected>>", self.ssd_updater.update)
        self.ssd_dropdown.menu.bind("<<ComboboxSelected>>", self.cooling_updater.update)
        self.cooling_dropdown.menu.bind("<<ComboboxSelected>>", self.fans_updater.update)
        self.fans_dropdown.menu.bind("<<ComboboxSelected>>", self.psu_updater.update)
        self.psu_dropdown.menu.bind("<<ComboboxSelected>>", self.case_updater.update)

        # Button to display selected components
        build_button = ttk.Button(self, text="Build PC", command=self.display_selected_components)
        build_button.grid(row=9, column=0, pady=10)

        # Text box to display selected components
        self.result_text = tk.Text(self, height=5, width=40)
        self.result_text.grid(row=10, column=0, pady=10)

    def display_selected_components(self):
        gpu = self.gpu_dropdown.var.get()
        cpu = self.cpu_dropdown.var.get()
        motherboard = self.motherboard_dropdown.var.get()
        ram = self.ram_dropdown.var.get()
        ssd = self.ssd_dropdown.var.get()
        cooling = self.cooling_dropdown.var.get()
        fans = self.fans_dropdown.var.get()
        psu = self.psu_dropdown.var.get()
        case = self.case_dropdown.var.get()
        result = f"GPU: {gpu}\nCPU: {cpu}\nMotherboard: {motherboard}\nRAM: {ram}\nSSD: {ssd}\nCooling: {cooling}\nFans: {fans}\nPSU: {psu}\nCase: {case}"
        self.result_text.delete(1.0, tk.END)
        self.result_text.insert(tk.END, result)

    def do_nothing(self, event):
        pass

def main():
    app = PCBuilderApp()
    app.mainloop()

if __name__ == "__main__":
    main()


# Graph-Based GUI

Voided since it is coding-heavy, but not very useful.